In [1]:
import netgen.gui
from ngsolve import *
from ngsolve.internal import visoptions
from netgen.meshing import ImportMesh

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2403
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 20 thread(s)


In [2]:
ngmesh = ImportMesh('cell_mesh_R1_opt.mesh')

In [3]:
from netgen.meshing import FaceDescriptor
isinplane = lambda el: all((((ngmesh[v][2] < -27) or (ngmesh[v][2] > 5)) for v in el.vertices))
fd_id = ngmesh.Add(FaceDescriptor(surfnr=2,domin=1,domout=0,bc=2))
for el in ngmesh.Elements2D():
    if isinplane(el):
        el.index = fd_id
ngmesh.SetBCName(fd_id-1, "new_bc")

In [4]:
mesh = Mesh(ngmesh)
Draw(mesh)

In [5]:
fes = VectorH1(mesh, order = 1, dirichlet="new_bc")
u,v = fes.TnT()

In [7]:
nsurf = specialcf.normal(3)
a = BilinearForm(fes, symmetric=True)
a += InnerProduct(grad(u).Trace(),grad(v).Trace())*ds
a += InnerProduct(grad(u).Trace() * nsurf, grad(v).Trace() * nsurf)*ds          
a.Assemble()  

# external force 
force = 0.01
nsurf = specialcf.normal(3)
f = LinearForm(fes)
# f += force*v*ds
f += InnerProduct(force*nsurf,v)*ds
f.Assemble()

SyntaxError: invalid non-printable character U+00A0 (2126232635.py, line 4)

Homogeneous Dirichlet data

In [8]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw(gfu, mesh, "u", autoscale=False)